In [6]:
import pandas as pd
import numpy as np
import os
import sys
import glob
from datetime import datetime
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)

from src.utils import choose_device


import matplotlib.pyplot as plt
import seaborn as sns

from src.fe_tools import create_time_features

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [5]:
config_file = './Sean/test_5_L10_pe.json'
config_file_name = config_file.split('/')[2].split('.')[0]
config_file_name

'test_5_L10_pe'

In [2]:
df_raw_data = pd.read_csv('../data/processed_data/combined_data.csv')
df_raw_data['datetime'] = pd.to_datetime(df_raw_data['datetime'])
# make sure the sorting is correct
df_raw_data = df_raw_data.sort_values(by=['device','datetime']).reset_index(drop=True)

df_device = choose_device(df_raw_data, 'L10')

In [15]:
# predicted result
dir = '../pred_results/Sean/'
result_file = [file for file in os.listdir(dir) if file.startswith(config_file_name)][0]
result_file

df_result = pd.read_csv(os.path.join(dir, result_file))

In [27]:
df_result.head()

,fold,datetime,y_valid,y_pred,tae
0,0,2024-06-30 07:24:12,12.16,14.809577,2.649577
1,0,2024-06-30 07:25:12,11.84,13.302484,1.462484
2,0,2024-06-30 07:26:12,11.14,11.424655,0.284655
3,0,2024-06-30 07:27:12,10.91,10.292706,0.617294
4,0,2024-06-30 07:28:12,10.54,10.299067,0.240933


In [ ]:
def get_target_data(df, error):
    if isinstance(error, (int, float)):
        df_temp = df[df['tae'] > error]

    elif isinstance(error, list) and len(error) == 2:
        df_temp = df[(df['tae'] >= error[0]) & (df['tae'] < error[1])]

    else:
        raise ValueError("Error should be either a single number or a list with two elements.")

    return df_temp

In [26]:
get_target_data(df_result, error=[500, 1000])

,fold,datetime,y_valid,y_pred,tae
395,0,2024-07-01 09:59:15,850.99,242.53152,608.458475
396,0,2024-07-01 10:00:15,799.59,143.06763,656.522373
397,0,2024-07-01 10:01:15,807.04,214.48730,592.552695
398,0,2024-07-01 10:02:15,819.33,224.87749,594.452513
399,0,2024-07-01 10:03:15,854.93,312.60733,542.322670
...,...,...,...,...,...
11790,4,2024-07-17 11:01:17,1522.96,1014.24603,508.713967
11903,4,2024-07-17 12:54:18,1517.96,822.37840,695.581582
12972,4,2024-07-19 12:57:22,1309.14,684.91230,624.227708
13898,4,2024-07-21 10:41:27,630.66,1216.30310,585.643101
